
<br>

<div style ="width: 100%;">
<table style ="width: 100%;background-color:float:center;">
<tr>

</tr>
    
<tr>
        
<td>
<div style="font-size: 21px;Margin-left:-10px;">
<b>Assinatura digital RSA<b>
</div>
    <br>
    <br>
<div style="font-size: 16px;">
    
Realizado por:

Carlos Ferreira a87953

Daniel Ribeiro a87994

João Nogueira a87973
</div>
</td>
<td >
<div style="display:inline-block; float:right;Margin-right:0px;Margin-top:20px;">
<img src="https://www.ecum.uminho.pt/SiteAssets/ECUM-PT.png" width="260" height="120">
<div>
</td>
</tr>
</table>
</div>
    

# Assinatura Digital RSA
No seguinte trabalho foi implementada a assinatura digital RSA, que consiste em uma técnica para a assinatura e validação de mensagens, visando a proteção da mesma. 
<br>
<br>
Suponhamos que a Alice deseja enviar uma mensagem assinada para o Bob. 
Ela pode usar a sua própria chave privada para o fazer, para isso produz um valor hash da mensagem, eleva-o à potência de d (chave privada) em módulo n:
<br>
$${\displaystyle s = m^{d}{\pmod{n}}}$$
<br>
e junta-o como uma "assinatura" à mensagem.
<br>
Quando o Bob recebe a mensagem assinada, ele usa o mesmo algoritmo de hash em conjunto com a chave pública da Alice. 
Ele depois eleva a assinatura à potência de e (módulo n):
<br>
$${\displaystyle m = s^{e}{\pmod {n}}}$$
<br>
e compara o valor hash resultante com o valor hash da mensagem. 
Se os dois coincidirem, ele sabe que o autor da mensagem foi a própria Alice ou alguém com a chave privada dela. 
<br>

<img style= "display: block;margin-left: auto;margin-right: auto;" src="https://image.slidesharecdn.com/rsasigrvb-190416012730/95/rsa-signature-verification-6-638.jpg?cb=1555378207" width="640" height="320">
<br>

De seguida iremos explicar como foi implementado este algoritmo em python, explicando cada função do mesmo suportado pela teoria por de trás disto tudo.



## Geração das chaves


Começamos por definir a função $\textbf{key_generation}$ onde geramos as chaves públicas e privadas para uma lista de nomes passada por parâmetro, e para cada nome da lista vamos aplicar o algoritmo de geração das chaves:

Começamos por gerar dois números primos grandes, distintos p e q.
    
Calculamos de seguida:
$${\displaystyle n=p\cdot q}$$ 

E após isso o lambda(n) que é o minimo multiplo comum de (p-1,q-1):
$${\lambda(n)=lcm(p-1,q-1)}$$ 

E escolhemos um valor de 'e' aleatório onde o máximo divisor comum entre ele e o lambda seja igual a 1.

Agora calculamos finalmente o d que vai ser a nossa chave privada tal que a seguinte equação seja verdade:

$${d ≡ e^{−1} (mod \space \lambda (n))}$$ 


Temos então a nossa chave privada a ser esse valor de d. 

E a nossa chave publica vai ser o par (n,e).


In [102]:
def key_generation(nomes):
    keys = {}
    for nome in nomes:
        p = random_prime(2^150 , 2^152)
        q = random_prime(2^148,2^149)
        n = p * q
        lambda_n =  lcm(p - 1, q - 1)
        e = randint ( 0 , lambda_n - 1 )
        while (gcd(e,lambda_n) != 1):
            e = randint ( 0 , e - 1 )
        d = inverse_mod(e,lambda_n)
        keys[nome] = {}
        keys[nome]['public_key'] = (n,e)
        keys[nome]['private_key'] = d
    return keys

Função auxiliar de print

In [103]:
#função auxiliar para impressão bonita do dicionário
def prettyPrint(d, indent=0):
      for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
             prettyPrint(value, indent+1)
        else:
             print('\t' * (indent+1) + str(value))



keys = key_generation(['Henrique','Daniel','João'])
prettyPrint (keys)

Henrique
	public_key
		(53790272515426648580258764901734561514275892348659038215202300086611683634263592210782029, 5434974425103743281596980630626112401935558430250450819545515469747234229698201146815963)
	private_key
		8093760433644170694497746839155971618920550253987795269920870960758705754057261580881683
Daniel
	public_key
		(180214499152099661662581706509166106697029400454132521113462319830719689095430705418651081, 3612980323771307843242574558098267652919509165743108940285932573867025847314563938811)
	private_key
		65210220957454997003417276696970091953699605552498082078933878856809151585178564571827
João
	public_key
		(12756942398837255917289105197402227561151423760566718512839841983378721554925478700753293, 3407815519684374481825323110770046364152785634313166617345120571305397660957440572325959)
	private_key
		2654527724475769205562007580314327643596690702474474523248092977233186394278558003349339


## Assinatura

Na nossa função de assinatura é onde assinamos a mensagem, vai ter como argumentos o dicionário das chaves, a mensagem e o nome de quem a vai enviar.

Começamos por calcular o hash da mensagem e após isso vamos assinar-la de acordo com a seguinte formula:

$${\displaystyle s = m^{d}{\pmod {n}}}$$

Sendo m o hash da mensagem, d a chave privada e n o primeiro componete da chave pública de quem envia.

Ficamos então em s com a assinatura, e damos retorno à mensagem junta com a sua assinatura.

In [101]:
import hashlib
def assinatura (keys, sender, mens):
    
  
    HH = hashlib.sha256(mens).hexdigest()
    HHH = ZZ('0x'+HH)
    d = keys[sender]['private_key']
    n = keys[sender]['public_key'][0]
    
    s = power_mod(HHH,d,n)
    

    print("Hash mensagem = ", HHH)
    print("Assinatura = ", s)
    return (mens,s)
    



## Validar assinatura

Na função valida vamos verificar se a mensagem foi mesmo assinada pela pessoa pretendida.

Como argumentos vamos precisar do nosso dicionário de chaves, a mensagem, a assinatura e o nome de quem enviou.

Depois de guardarmos ambas as componentes da chave pública de quem fez o envio em 'n' e 'e' vamos passar á validação da assinatura.

Como na assinatura da mesma, vamos fazer hash da mensagem e guardar esse hash em m1.

E vamos utilizar a seguinte formula na assinatura:

$${\displaystyle m2 = s^{e}{\pmod {n}}}$$

Caso este m2 agora calculado for igual ao m1 calculado antes, a mensagem e a assinatura correspondem, caso contrário não correspondem á mesma pessoa.

In [100]:
    
def valida(keys,m,s,sender):
    
    
    n = keys[sender]['public_key'][0]
    e = keys[sender]['public_key'][1]
    
        
    HH = hashlib.sha256(m).hexdigest()
    m1 = ZZ('0x'+HH)
    print("m1 = ",m1)
    
    
    m2 = power_mod(s,e,n)
    print("m2 = ",m2)
    
    
    return m1 == m2

# Exemplos
Agora com uns pequenos exemplos vamos testar o nosso sistema:

In [104]:
m,s = assinatura (keys , 'Henrique', b"Ola Joao")
v = valida(keys,m,s,'Henrique')
print("O m1 e o m2 correspondem" if v else "O m1 e o m2 não correspondem")

Hash mensagem =  26597243779506947528044713433246156167874695102361963663476175799745301648860
Assinatura =  25486871129426990094614540689550836812959157294305548697237799623990554481452660785352863
m1 =  26597243779506947528044713433246156167874695102361963663476175799745301648860
m2 =  26597243779506947528044713433246156167874695102361963663476175799745301648860
O m1 e o m2 correspondem


In [105]:
m,s = assinatura (keys , 'João', b"Ola Henrique")
v = valida(keys,m,s,'João')
print("O m1 e o m2 correspondem" if v else "O m1 e o m2 não correspondem")

Hash mensagem =  115672307424039632516976815098144454233709661282371491025542307220040351831551
Assinatura =  9141143681988754312341673816690326655344738873692168233169117483976253085770030461995176
m1 =  115672307424039632516976815098144454233709661282371491025542307220040351831551
m2 =  115672307424039632516976815098144454233709661282371491025542307220040351831551
O m1 e o m2 correspondem


Como podemos verificar a mensagem e a assinatura corresponderam, desta forma podemos verificar que o nosso sistema está a funcionar.

# Assinatura cega de Chaum

Existem esquemas de assinatura cega para muitos métodos de assinatura de chave pública. Mais formalmente, um esquema de assinatura cega é um protocolo criptográfico que envolve duas partes, uma utilizadora Alice que deseja obter assinaturas nas suas mensagens e um assinante Bob. 
No final do processo, a Alice obtém a assinatura do Bob na sua mensagem sem que o Bob saiba do conteúdo da mensagem.

<br>

<img style= "display: block;margin-left: auto;margin-right: auto;margin-top: 20px;" src="https://cdn.discordapp.com/attachments/821737859592880141/838785203572441108/q.png" width="480" height="270">

<br>


# Blinding

Uma assinatura RSA tradicional é calculada elevando a mensagem m à chave privada d módulo chave pública n. 
A versão cega usa um valor aleatório r, tal que 0 < r < n, de modo que r é primo relativo a n (ou seja, mdc (r, n) = 1) , r é elevado ao expoente público e módulo n:

$${\displaystyle m' = m \cdot r^{e}{\pmod {n}}}$$

e envia o valor resultante m' para quem vai assinar a mensagem. 
Como r é um valor aleatório e como $${\displaystyle r^{e} {\bmod {n}}}$$ é uma permutação, segue-se que $${\displaystyle r^{e} {\bmod {n}}}$$ também é aleatório. 
Isso implica que m' não contém informação sobre m, assim quem vai assinar a mensagem não tem conhecimento sobre a mensagem m.

In [99]:
def encriptarBlind (keys , sender , signer, mens):
    
    
    
    HH = hashlib.sha256(mens).hexdigest()
    m = ZZ('0x'+HH)
    n = keys[signer]['public_key'][0]
    e = keys[signer]['public_key'][1]
    d = keys[signer]['private_key']
    
    
    r = randint(0,n)
    
    while (gcd(r,n) != 1):
        r = randint(0,n)
    
    
    m_linha = mod(m,n) * (power_mod(r,e,n))
    
    print ("r = ",r,"\n")
    print ("m' = ",m_linha,"\n")
    
    return m_linha,r,m

# Assinatura
De seguida vamos criar a assinatura cega de acordo com a seguinte formula:

$${\displaystyle s' = m'^{d}{\pmod {n}}}$$

Esta assinatura (s') é depois enviada ao autor da mensagem.

In [109]:
def assinaturaBlind (keys,m_linha,sender,signer):
    
    d = keys[signer]['private_key']
    n = keys[signer]['public_key'][0]
    
        
    s_linha = power_mod(m_linha,d,n)
    print ("s' = ",s_linha,"\n")
    
    
    return s_linha

# Validar Assinatura

Finalmente o autor da mensagem vai validar a assinatura na mensagem m', para isso a assinatura após a remoção do <i>blinding factor</i> (s1)

$${\displaystyle s1 = s' \cdot r^{-1} \pmod {n}}$$

E a assinatura de quem assinou m', na mensagem m (s2) têm de igualar
 
$${\displaystyle s2 = m^{d}{\pmod {n}}}$$

Logo se $s1 = s2$ a assinatura cega é válida.



In [97]:
def validaAssinatura (keys,s_linha,signer,sender,r,m):
    
    n = keys[signer]['public_key'][0]
    d = keys[signer]['private_key']
    
        
    s1 = mod(s_linha,n) * (power_mod(r,-1, n ))
    
    s2 = power_mod(m,d,n)
    
    print("s1 = ",s1,"\n")
    print("s2 = ",s2,"\n")
    
    return s1 == s2

# Exemplos

Com uns pequenos exemplos vamos testar o nosso sistema:

In [118]:
s,r,m = encriptarBlind (keys , 'Henrique' , 'Daniel',b"Ola Daniel")
ss = assinaturaBlind (keys , s , 'Henrique' , 'Daniel')
sss = validaAssinatura (keys , ss , 'Daniel' , 'Henrique',r,m)

print("S1 e S2 correspondem" if sss else "S1 e S2  não correspondem")

r =  16987078054790715822524241868153274478029463262091435444671746523078722744225387843640671 

m' =  23042875446747653178641636783374705380930038928534344040849360322937790630843976449385333 

s' =  54408755726430950281950315085733445928093752419957610448784109540544990891356365354005478 

s1 =  49663305704380057633249475224962420256549497711906209164374222170937776404050414555761067 

s2 =  49663305704380057633249475224962420256549497711906209164374222170937776404050414555761067 

S1 e S2 correspondem


In [119]:
s,r,m = encriptarBlind (keys , 'Henrique' , 'João',b"Ola Joao")
ss = assinaturaBlind (keys , s , 'Henrique' , 'João')
sss = validaAssinatura (keys , ss , 'Daniel' , 'Henrique',r,m)

print("S1 e S2 correspondem" if sss else "S1 e S2  não correspondem")

r =  2303036650718789017536112240858638501516402495868695255452267319886562283833137143625668 

m' =  7338936122409252535844631972327498302246055795915616077788004469698773984923780628695088 

s' =  10513471178563957547026354340663660101529259223445132177425342629937404541862898834115119 

s1 =  127899416007523708408306293785391770679301672374504585295445505998604359806602132736150276 

s2 =  29159872200243095489258132446139254438284824254633767725758123568009788592631560748469941 

S1 e S2  não correspondem


Assim como podemos verificar S1 e S2 correspondem logo o sistema de assinatura cega está a funcionar.